### 🧭 설치 및 준비 작업
Voyage API를 사용하기 위해 필요한 패키지를 설치합니다.

In [ ]:
%pip install voyageai

### 🔑 Voyage API 키 설정 및 클라이언트 생성

In [ ]:
from dotenv import load_dotenv  
import voyageai

load_dotenv()
client = voyageai.Client()

### 📄 PDF에서 텍스트 + 이미지 추출 (LangChain + PyPDF 사용)

In [ ]:
from langchain.document_loaders import PyPDFLoader
import fitz
import os

# PDF 파일 경로 설정
pdf_path = "./data/국가별 공공부문 AI 도입 및 활용 전략.pdf"

# 텍스트 추출: LangChain 활용
loader = PyPDFLoader(pdf_path)
documents = loader.load()
texts = [doc.page_content for doc in documents]

# 이미지 추출: fitz 활용
os.makedirs("./data/pdf_image_voyage", exist_ok=True)
doc = fitz.open(pdf_path)
image_paths = []

for i, page in enumerate(doc):
    pix = page.get_pixmap()
    img_path = f"./data/pdf_image_voyage/{i}.png"
    pix.save(img_path)
    image_paths.append(img_path)

doc.close()

### 📦 텍스트와 이미지를 하나의 시퀀스로 구성

In [ ]:
from PIL import Image

inputs = []
for text, image_path in zip(texts, image_paths):
    try:
        image = Image.open(image_path)
        inputs.append([text, image])
    except Exception as e:
        print(f"이미지 로드 실패: {image_path} => {e}")

### 📐 Voyage API로 멀티모달 임베딩 생성

In [ ]:
result = client.multimodal_embed(
    inputs=inputs,
    model="voyage-multimodal-3",
    input_type="document"
)

embeddings = result.embeddings
print(f"총 벡터 개수: {len(embeddings)}")

### 🔍 질의 텍스트와 이미지 임베딩 생성 (검색용 쿼리)

In [ ]:
query = ["스마트폰을 들고 있는 사람"]
query_result = client.multimodal_embed(
    inputs=[query],
    model="voyage-multimodal-3",
    input_type="query"
)

query_vector = query_result.embeddings[0]

### 🧠 Qdrant를 활용한 멀티모달 벡터 저장 및 검색

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import uuid

# Qdrant 메모리 기반 클라이언트 생성
qdrant_client = QdrantClient(":memory:")
collection_name = "voyage-multimodal-demo"

# Qdrant 컬렉션 생성
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=1024,  # Voyage 모델의 출력 벡터 차원
        distance=models.Distance.COSINE
    ),
    on_disk_payload=True
)

# Qdrant에 문서 벡터 업로드
points = []
for i, vector in enumerate(embeddings):
    points.append(
        models.PointStruct(
            id=str(uuid.uuid4()),
            vector=vector,
            payload={"source": "voyage", "text": texts[i],"image_path": image_paths[i]}
        )
    )

qdrant_client.upsert(collection_name=collection_name, points=points)
print(f"{len(points)}개의 문서를 Qdrant에 업로드 완료.")


In [ ]:
# Qdrant를 통한 질의 벡터 검색
query = [Image.open(image_paths[5])]
query_result = client.multimodal_embed(
    inputs=[query],
    model="voyage-multimodal-3",
    input_type="query"
)

query_vector = query_result.embeddings[0]


search_result = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=5
)

for rank, hit in enumerate(search_result):
    print(f"{rank+1}위: 문서 ID {hit.id}, 유사도 점수: {hit.score:.4f}")

In [ ]:
Image.open(image_paths[5])

In [ ]:
image_path = search_result[0].payload['image_path']

print(f"이미지 경로: {image_path}")

# 이미지 시각화
import matplotlib.pyplot as plt
from PIL import Image

# 이미지 불러오기 및 표시
img = Image.open(image_path)
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')
plt.title(f"Image ID: {image_path}")
plt.show()